In [1]:
import math
import random
random.seed(123)
import numpy as np
import pandas as pd
np.random.seed(123)

In [2]:
# read txt file and convert it into pandas dataframe.
txt = ''
with open('seeds_dataset.txt') as f :
    txt = f.read()
    

In [3]:
data = []
for line in txt.split('\n') :
    data.append([word for word in line.split('\t') if word != ''])

In [4]:
df = pd.DataFrame(data,columns=['area','perimeter','compactness','kernel_len','kernel_wid','coef_asymmetry','kernel_groove_len'
                               ,'wheat_type'],copy=True)

In [5]:
df.wheat_type.unique()

array(['1', '2', '3'], dtype=object)

In [6]:
df.shape

(210, 8)

In [7]:
def get_normal_random_weight() :
    return random.uniform(0, 1)
get_normal_random_weight()

0.052363598850944326

In [8]:
def build_single_neuron(inp) :
    neuron = {}
    neuron['weights'] = [ random.uniform(0, 1) for i in range(inp+1) ]
    return neuron

In [9]:
def build_single_hidden_layer(no_of_neurons,no_of_inputs) :
    layer = []
    for i in range(no_of_neurons) :
        layer.append(build_single_neuron(no_of_inputs))
    return layer

In [10]:
def build_hidden_layers(layers,no_of_neurons,no_of_inputs) :
    network = []
    for i in range(layers) :
        network.append(build_single_hidden_layer(no_of_neurons,no_of_inputs))
    return network

In [11]:
def build_output_layer(out_neuron,no_of_inputs) :
    return build_single_hidden_layer(out_neuron,no_of_inputs)

In [12]:
def calculate_activation_value(weights,inputs) :
    activation = weights[-1]
    for i in range(len(weights)-1) :
        activation += weights[i] * float(inputs[i])
    return activation

In [13]:
def transfer(act) :
    return 1.0/(1.0+math.exp(-act))

In [14]:
def forward_propagate(network,row) :
    inputs = row
    for i,layer in enumerate(network) :
        new_inputs = []
        for j,neuron in enumerate(layer) :
            activation = calculate_activation_value(neuron['weights'],inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [15]:
inp = df.shape[1] -1 
hidden_layers = 2    # two hidden layers.
hidden_neurons = 7 

out = len(df.wheat_type.unique())
print(inp,hidden_layers,hidden_neurons,out)

def build_neural_network(inp,hidden_layers,hidden_neurons,out) :
    network = build_hidden_layers(hidden_layers,hidden_neurons,inp)
    network.append(build_output_layer(out,hidden_neurons))
    return network


7 2 7 3


In [16]:
def transfer_derivatives(out) :
    return out*(1-out)

In [19]:
def backward_propagate_error(network,expected) :
    for i in reversed(range(len(network))) :
        layer = network[i]
        errors = []
        if i != len(network)-1 :
            for j in range(len(layer)) :
                error = 0.0
                for neuron in network[i+1] :
                    error += (neuron['weights'][j]*neuron['delta'])
                errors.append(error)
        else :
            for j in range(len(layer)) :
                neuron = layer[j]
                errors.append(float(expected[j])-neuron['output'])
        for j in range(len(layer)) :
            neuron = layer[j]
            neuron['delta'] = errors[j]*transfer_derivatives(neuron['output']) 

In [30]:
def update_weights(network,row,lr=0.1) :
    for i in range(len(network)) :
        inputs = row[:-1]
        if i != 0 :
            inputs = [ neuron['output'] for neuron in network[i-1] ]
        for neuron in network[i] :
            for j in range(len(inputs)) :
                neuron['weights'][j] += lr*float(neuron['delta'])*float(inputs[j])
            neuron['weights'][-1] += lr*neuron['delta'] 

In [32]:
# do a forward pass for all rows.
expected = [0, 1, 2]
def single_pass() :
    network = build_neural_network(inp,hidden_layers,hidden_neurons,out)
    for i in range(df.shape[0]) :
        forward_propagate(network,df.iloc[i].tolist())
        backward_propagate_error(network,df['wheat_type'].tolist())
        update_weights(network,df.iloc[i].tolist())
#forward_pass()

In [35]:
def train_network(epochs) :
    for i in range(epochs) :
        single_pass()
train_network(10)

In [ ]:
# adding keras for comparison
# from keras.models import Sequential
# from keras.layers import Dense, Activation
# model = Sequential()
# model.add(Dense(32, input_dim=784))
# model.add(Activation('relu'))
